In [1]:
import os
import torch

from tqdm.notebook import tqdm
%matplotlib notebook 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80


from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from src.mesh.cube import NetCube, to_vertices
from argparse import Namespace


device = torch.device('cuda')

In [2]:
# Load the dolphin mesh.
trg_obj = os.path.join('./data/scenes/centered/meshes/centered.obj')

# We read the target 3D model using load_obj
verts, faces, aux = load_obj(trg_obj)

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

# We scale normalize and center the target mesh to fit in a sphere of radius 1 centered at (0,0,0). 
# (scale, center) will be used to bring the predicted mesh to its original center and scale
# Note that normalizing the target mesh, speeds up the optimization but is not necessary!
center = verts.mean(0)
verts = verts - center
scale = max(verts.abs().max(0)[0])
verts = verts / scale

# We construct a Meshes structure for the target mesh
trg_mesh = Meshes(verts=[verts], faces=[faces_idx])#.to(device)
trg_mesh

In [3]:
opt = Namespace(
    n = 82,
    nfc=256, 
    min_nfc=256, 
    ker_size=3,
    num_layer=5,
    stride=1,
    padd_size=0,
    nc_im=3,
)

n = 32
cube = NetCube(n, opt, kernel=7, sigma=2).cuda()
cube.load_state_dict(torch.load('./data/net_fitted_32.pth'))

#radii = torch.rand(3, device=v_ref.device, requires_grad=True)
v, f = cube()
# n_ref = compute_face_normals(v_ref, f_ref)
print([f.shape for f in [v, f]])

src_mesh =  Meshes(verts=[cube.get_start().to(device)], faces=[f])
src_mesh

[torch.Size([6144, 3]), torch.Size([12284, 3])]


In [4]:
cube.get_start().shape, v.shape

(torch.Size([6144, 3]), torch.Size([6144, 3]))

In [5]:
sample_trg = sample_points_from_meshes(trg_mesh, 5000)
new_src_mesh = src_mesh.offset_verts(v)
sample_src = sample_points_from_meshes(new_src_mesh, 5000)
sample_trg, sample_src

(tensor([[[ 0.1600, -0.6036,  0.2901],
          [-0.2708, -0.4012, -0.2900],
          [ 0.1964, -0.7290, -0.0019],
          ...,
          [ 0.8085,  0.0197,  0.3128],
          [ 0.8249, -0.1524,  0.2105],
          [ 0.4368,  0.8063, -0.3777]]], device='cuda:0'),
 tensor([[[ 0.8071, -0.0321, -0.2795],
          [ 0.8036,  0.6846,  0.2456],
          [ 0.3370,  0.2591, -0.5222],
          ...,
          [ 0.6695, -0.6024, -0.0213],
          [ 0.3162,  0.5422, -0.5035],
          [-0.1769,  0.6618, -0.3012]]], device='cuda:0',
        grad_fn=<IndexPutBackward0>))

In [ ]:
#optimizer = torch.optim.SGD(cube.parameters(), lr=1.0, momentum=0.9)
optimizer = torch.optim.Adam(cube.parameters(), lr=0.0003)
optimizer

In [ ]:
# Number of optimization steps
Niter = 20000
# Weight for the chamfer loss
w_chamfer = 1.0 
# Weight for mesh edge loss
w_edge = 1.0 
# Weight for mesh normal consistency
w_normal = 0.01 
# Weight for mesh laplacian smoothing
w_laplacian = 0.1 
# Plot period for the losses
plot_period = 250
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []



for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()
    
    v, _ = cube()
    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(v)
    
    # We sample 5k points from the surface of each mesh 
    sample_trg = sample_points_from_meshes(trg_mesh, 25000)
    sample_src = sample_points_from_meshes(new_src_mesh, 25000)
    
    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    
    #if False:
    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian
    #loss = loss_chamfer
    # Print the losses
    loop.set_description('total_loss = %.6f' % loss)
    
    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
#     edge_losses.append(float(loss_edge.detach().cpu()))
#     normal_losses.append(float(loss_normal.detach().cpu()))
#     laplacian_losses.append(float(loss_laplacian.detach().cpu()))
    
    # Plot mesh
    #if i % plot_period == 0:
    #    plot_pointcloud(new_src_mesh, title="iter: %d" % i)
        
    # Optimization step
    loss.backward()
    optimizer.step()
    
chamfer_losses    

In [ ]:
import meshplot


In [ ]:
vert =  cube.get_start().to(device) + v
meshplot.plot(vert.detach().cpu().numpy(), f.cpu().numpy())

In [ ]:
vert